In [1]:
import flask

# Chapter 1: Hello, World!

### Hello World flask application.

The application will exist in a package. In Python, a sub-directory that includes a __init.py__ files is considered a package, and can be imported. When you import a package, the __init__.py executes and defines what symbols the package exposes to the outside world.

The __init__.py created in the folder `app` has the following code:

```python
from flask import Flask
app = Flask (__name__)

from app import routes
```

The script above simply creates the application object as an instance of class `Flask` class is a Python predefined variable, which is set to the name of the module in which it is used. Flask uses the location of the module passed here as a starting point when it needs to load associated resources such as template files. For all practical purposes, passing `__name__` is almost always going to configure Flask in the correct way. The application then imports `routes`, which doesn't exist yet. 

`routes` are imported at the bottom, and not at the top. The bottom import is a workaround to circular imports, a common problem with Flask applications. `routes` module will need to import the `app` variable defined in this script, so putting one of the reciprocal imports at the bottom avoids the error that results from the mutual references between these two files.

The routes are the different URLs that the application implements. In Flask, handlers for the application routes are written as Python functions, called view functions; it takes a web request and returns a web response. 

Content of `routes.py`:

```python
from app import app

@app.route('/')
@app.route('/index')
def index():
    return "Hello, World!"
```
Two `@app.route` above the function are decorators, a unique feature in Python. A decorator modifies the function that follows it. A common pattern with decorators is to use them to register functions as callbacks for certain events. In this case, the `@app.route` decorator creates an association between the URL given as an argument and the function. Here, there are two decorators, and when a web browser requests either of these two URLs, Flask is going to invoke this function and pass the return value of it back to the browser as a response.  

Both `routes.py` and `__init.py__` are saved in the folder named `app`. The third function, `microblog.py` is defined in the main folder. The content:
```python
from app import app
```

Before we run it, Flask needs to be told how to import it, by setting the `FLASK_APP=microblog.py`. `FLASK_APP` is used to specify how to load application. 

Note! It seems like there are two ways to run it: one with `FLASK_app`; the other one is to define `if __name__ = main`

Since environment variabels aren't remembered across terminal sessions, it's annoying to always set the `FLASK_APP` when you open a new terminal window. We can register environmental variables that we want to import when we run the `flask` command. For that, `python-dotenv` is used

# Chapter 2: Templates

Just for the sake of discussion, we want to have a heading that welcomes the user. A dummy user can be created via Python in the form of dictionary:

```python
user = {'username': 'Miguel'}
```
The view function in the application returns a simple string. We want to expand that string into a complete HTML page (`routes.py` will be modified):

```python
from app import app

@app.route('/')
@app.route('/index')
def index():
    user = {'username': 'Miguel'}
    return '''
<html>
    <head>
        <title>Home Page - Microblog</title>
    </head>
    <body>
        <h1>Hello, ''' + user['username'] + '''!</h1>
    </body>
</html>'''
```
However, this is not the good approach. The code will become extremely complex when we add new posts from users. Application will also have more view functions that are going to be associated with other URLs, so if one day the layout of the application will be modified, the HTML in every view will have to be updated. The solution is to keep the logic of the application from the layout of the web pages. Templates help achieve this separation between the presentation and business logic.

The resulting code is placed into `app/templates/index.html`

```html
<html>
    <head>
        <title>{{ title }} - Microblog</title>
    </head>
    <body>
        <h1>Hello, {{ user.username }}!</h1>
    </body>
</html>
```

There are couple of placheolders, enclosed in `{{}}`. They represent the parts of the page that are variable and will only be known at runtime.

`routes.py` is modified to:
```python
from flask import render_template
from app import app

@app.route('/')
@app.route('/index')
def index():
    user = {'username': 'Jony'}
    return render_template ('index.html', title='Home', user=user)
```

Question: How does routes find the html? They are in different folders.
Answer: Flask looks for `templates` folder. `Flask is looking in templates/frontend/src/view_notifications.html for your template file` (source: https://stackoverflow.com/questions/31002890/how-to-reference-a-html-template-from-a-different-directory-in-python-flask/31003097)

The operation that converts a template into a complete HTML page is called rendering. The imported function invokes Jinja2 template engine and Jinja2 substitutes `{{...}}` with the corresponding values

#### Conditional Statements

Templates also support control statements, given inside `{%...%}` blocks.
```html
<html>
    <head>
        {% if title %}
        <title>{{ title }} - Microblog</title>
        {% else %}
        <title>Welcome to Microblog!</title>
        {% endif %}
    </head>
    <body>
        <h1>Hello, {{ user.username }}!</h1>
    </body>
</html>
```

If title is not passed, then instead of empty title, we get another string.

#### Loops

Changing `routes.py` to 

```python
from flask import render_template
from app import app

@app.route('/')
@app.route('/index')
def index():
    user = {'username': 'Miguel'}
    posts = [
        {
            'author': {'username': 'John'},
            'body': 'Beautiful day in Portland!'
        },
        {
            'author': {'username': 'Susan'},
            'body': 'The Avengers movie was so cool!'
        }
    ]
    return render_template('index.html', title='Home', user=user, posts=posts)
```
